In [5]:
import math
from collections import Counter
import pandas as pd

def calculate_entropy(data, target_column):
    class_counts = Counter([row[target_column] for row in data])
    total_rows = len(data)
    entropy = 0.0
    for count in class_counts.values():
        probability = count / total_rows
        entropy -= probability * math.log2(probability)
    return entropy

def calculate_information_gain(data, attribute, target_column):
    total_entropy = calculate_entropy(data, target_column)
    attribute_values = set([row[attribute] for row in data])
    weighted_entropy = 0.0
    total_rows = len(data)
    for value in attribute_values:
        subset = [row for row in data if row[attribute] == value]
        subset_size = len(subset)
        subset_entropy = calculate_entropy(subset, target_column)
        weighted_entropy += (subset_size / total_rows) * subset_entropy
    information_gain = total_entropy - weighted_entropy
    return information_gain

def find_best_attribute(data, attributes, target_column):
    best_attribute = None
    max_gain = -1
    for attribute in attributes:
        gain = calculate_information_gain(data, attribute, target_column)
        print(f"Information Gain for {attribute}: {gain:.4f}")
        if gain > max_gain:
            max_gain = gain
            best_attribute = attribute
    return best_attribute

def build_decision_tree(data, attributes, target_column, tree=None):
    if tree is None:
        tree = {}
    classes = [row[target_column] for row in data]
    if len(set(classes)) == 1:
        return classes[0]
    if not attributes:
        majority_class = Counter(classes).most_common(1)[0][0]
        return majority_class
    best_attribute = find_best_attribute(data, attributes, target_column)
    print(f"\nBest attribute to split on: {best_attribute}\n")
    tree = {best_attribute: {}}
    remaining_attributes = [attr for attr in attributes if attr != best_attribute]
    attribute_values = set([row[best_attribute] for row in data])
    for value in attribute_values:
        subset = [row for row in data if row[best_attribute] == value]
        if not subset:
            majority_class = Counter(classes).most_common(1)[0][0]
            tree[best_attribute][value] = majority_class
        else:
            subtree = build_decision_tree(subset, remaining_attributes, target_column)
            tree[best_attribute][value] = subtree
    return tree

def classify(tree, instance):
    if isinstance(tree, str):
        return tree
    attribute = next(iter(tree))
    attribute_value = instance.get(attribute)
    if attribute_value not in tree[attribute]:
        return None
    subtree = tree[attribute][attribute_value]
    return classify(subtree, instance)

def print_rules(tree, rule=""):
    if isinstance(tree, str):
        print(f"IF {rule} THEN Class = {tree}")
        return
    attribute = next(iter(tree))
    for value, subtree in tree[attribute].items():
        new_rule = f"{rule} AND {attribute} = {value}" if rule else f"{attribute} = {value}"
        print_rules(subtree, new_rule)

def print_tree(tree, indent=""):
    if isinstance(tree, str):
        print(indent + "└── Class:", tree)
        return
    attribute = next(iter(tree))
    print(indent + attribute)
    for value, subtree in tree[attribute].items():
        print(indent + "├──", value)
        print_tree(subtree, indent + "│   ")

def load_csv_data(file_path, target_column):
    df = pd.read_csv(file_path)
    if target_column not in df.columns:
        raise ValueError(f"Target column '{target_column}' not found in dataset.")
    attributes = [col for col in df.columns if col != target_column]
    data = df.to_dict(orient='records')
    return data, attributes, target_column

def decision_tree_main():
    file_path = input("Enter the path to the CSV file: ").strip()
    df = pd.read_csv(file_path)
    print("\nAvailable columns:", ', '.join(df.columns))
    target_column = input("Enter the name of the target column: ").strip()
    data, attributes, target_column = load_csv_data(file_path, target_column)

    print("\nStep 1: Calculating Initial Entropy (Info(D))")
    initial_entropy = calculate_entropy(data, target_column)
    print(f"Initial Entropy (Info(D)): {initial_entropy:.4f}\n")

    print("Step 2: Calculating Information Gain for Each Attribute")
    for attribute in attributes:
        gain = calculate_information_gain(data, attribute, target_column)
        print(f"Information Gain for {attribute}: {gain:.4f}")

    print("\nStep 3: Building the Decision Tree")
    decision_tree = build_decision_tree(data, attributes, target_column)

    print("\nStep 4: Decision Tree Rules")
    print_rules(decision_tree)

    print("\nStep 5: Text-Based Tree Visualization")
    print_tree(decision_tree)

# Run the decision tree main function
decision_tree_main()


Enter the path to the CSV file: /content/plays_football.csv

Available columns: Outlook, Temperature, Humidity, Windy, Class
Enter the name of the target column: Class

Step 1: Calculating Initial Entropy (Info(D))
Initial Entropy (Info(D)): 0.9403

Step 2: Calculating Information Gain for Each Attribute
Information Gain for Outlook: 0.2467
Information Gain for Temperature: 0.0292
Information Gain for Humidity: 0.1518
Information Gain for Windy: 0.0161

Step 3: Building the Decision Tree
Information Gain for Outlook: 0.2467
Information Gain for Temperature: 0.0292
Information Gain for Humidity: 0.1518
Information Gain for Windy: 0.0161

Best attribute to split on: Outlook

Information Gain for Temperature: 0.0200
Information Gain for Humidity: 0.0200
Information Gain for Windy: 0.9710

Best attribute to split on: Windy

Information Gain for Temperature: 0.5710
Information Gain for Humidity: 0.9710
Information Gain for Windy: 0.4200

Best attribute to split on: Humidity


Step 4: Decisi